Данная задача полностью аналогична предыдущей, но проверять те-
перь нужно более общее свойство. Дереву разрешается содержать
равныеключи,ноонивсегдадолжнынаходитьсявправомподдереве.
Формально, двоичное дерево называется деревом поиска, если для
любой вершины её ключ больше всех ключей из её левого поддерева
и **не меньше** всех ключей из правого поддерева.

In [1]:
%matplotlib inline
from IPython.display import Image, HTML
import networkx as nx

In [2]:
class TreeNode:
    def __init__(self):
        self.key = self.left = self.right = None


def parse_tree(lines):
    if isinstance(lines, str):
        lines = lines.split(';') if lines else []
    tree = [TreeNode() for _ in range(len(lines))]
    for num, line in enumerate(lines):
        if isinstance(line, str):
            key, left, right = map(int, line.strip().split())
        tree[num].key = key
        tree[num].num = num
        if left != -1:
            tree[num].left = tree[left]
        if right != -1:
            tree[num].right = tree[right]
    return tree[0] if tree else None


def draw_tree(tree, root=0,
              width=.07, vgap=.05, vloc=0, xcenter=.5,
              node_color='lightgray', node_size=800,
              **kw):
    if isinstance(tree, str):
        tree = parse_tree(tree)
    if tree is None:
        return

    def add_node(node):
        num = node.num
        if num not in graph.nodes:
            graph.add_node(num)
            labels[num] = str(node.key)
        left = node.left
        if left is not None:
            add_node(left)
            graph.add_edge(num, left.num)
            labels[left.num] += '<'
        right = node.right
        if right is not None:
            add_node(right)
            graph.add_edge(num, right.num)
            labels[right.num] += '>'

    graph = nx.Graph()
    labels = {}
    add_node(tree)

    def layout_hier(root, width, vloc, xcenter, pos=None, parent=None):
        if pos is None:
            pos = { root: (xcenter, vloc) }
        else:
            pos[root] = (xcenter, vloc)
        neighbors = list(graph.neighbors(root))
        if parent is not None:
            neighbors.remove(parent)
        if len(neighbors):
            dx = width / len(neighbors) 
            nextx = xcenter - width/2 - dx/2
            for neighbor in neighbors:
                nextx += dx
                pos = layout_hier(neighbor, width=dx, vloc=vloc-vgap, xcenter=nextx,
                                  pos=pos, parent=root)
        return pos

    pos = layout_hier(root, width, vloc, xcenter)
    return nx.draw(graph, pos=pos, with_labels=True, labels=labels,
                   node_color=node_color, node_size=node_size,
                   **kw)

In [3]:
def test_validate2(validate):
    tree = parse_tree('2 1 2;1 -1 -1;3 -1 -1')
    assert validate(tree) == True

    tree = parse_tree('1 1 2;2 -1 -1;3 -1 -1')
    assert validate(tree) == False

    tree = parse_tree('2 1 2;1 -1 -1;2 -1 -1')
    assert validate(tree) == True

    tree = parse_tree('2 1 2;2 -1 -1;3 -1 -1')
    assert validate(tree) == False

    tree = parse_tree('2147483647 -1 -1')
    assert validate(tree) == True
    
    tree = parse_tree('30 1 2;20 -1 -1;50 3 4;40 -1 -1;60 -1 -1')
    assert validate(tree) == True
    
    tree = parse_tree('30 1 2;20 -1 -1;50 3 4;30 -1 -1;60 -1 -1')
    assert validate(tree) == True
    
    tree = parse_tree('30 1 2;20 -1 -1;50 3 4;29 -1 -1;60 -1 -1')
    assert validate(tree) == False
    
    print('ok')

In [4]:
def validate_recur_postorder(tree):
    def recurse(node):
        cur_val = cur_min = cur_max = node.key
        if node.left:
            left_min, left_max = recurse(node.left)
            assert cur_val > left_max
            cur_min = min(cur_min, left_min)
            cur_max = max(cur_max, left_max)
        if node.right:
            right_min, right_max = recurse(node.right)
            assert cur_val <= right_min
            cur_min = min(cur_min, right_min)
            cur_max = max(cur_max, right_max)
        return cur_min, cur_max
    try:
        if isinstance(tree, str):
            tree = parse_tree(tree)
        if tree:
            recurse(tree)
        return True
    except AssertionError:
        return False

test_validate2(validate_recur_postorder)

ok


#### Python без рекурсии.
Используем стек текущей точки в дереве с поддержкой максимума/минимума.
Пока ходим по дереву, всё время копим левую и правую границы допустимости
(вначале они не определены).
Шагая _влево_, уточняем _правую_ границу и сохраняем текущую точку+границы на стеке,
чтобы впоследствии продолжить из неё движение, но уже вправо.
Когда левые шаги исчерпались, вынимаем сохранённую точку+границы из стека
и шагаем _вправо_, уточняя при этом _левую_ границу.

In [5]:
def validate_iter_inorder(tree):
    node = parse_tree(tree) if isinstance(tree, str) else tree
    stack = []
    lval = rval = None
    valid = True
    while node or stack:
        if node:
            stack.append((node, lval, rval))
            key = node.key
            valid = ((lval is None or lval <= key) and
                     (rval is None or key < rval))
            if not valid:
                return False
            rval = key if rval is None else min(rval, key)
            node = node.left
            continue
        node, lval, rval = stack.pop()
        key = node.key
        lval = key if lval is None else max(lval, key)
        node = node.right
    return valid

test_validate2(validate_iter_inorder)

ok


In [6]:
%%writefile tree-validity2.py
#########
# exam

from collections import namedtuple
Node = namedtuple('Node', ['key', 'left', 'right'])

num = int(input())
tree = [Node(*map(int, input().split())) for _ in range(num)]

node = 0 if tree else -1
stack = []
lval = rval = None
valid = True

while node != -1 or stack:
    if node != -1:
        stack.append((node, lval, rval))
        key = tree[node].key
        valid = ((lval is None or lval <= key) and
                 (rval is None or key < rval))
        if not valid:
            break
        rval = key if rval is None else min(rval, key)
        node = tree[node].left
        continue
    node, lval, rval = stack.pop()
    key = tree[node].key
    lval = key if lval is None else max(lval, key)
    node = tree[node].right

print(['INCORRECT','CORRECT'][valid])

Writing tree-validity2.py


In [7]:
%%bash
python3 tree-validity2.py << EOF
3
2 1 2
1 -1 -1
3 -1 -1
EOF
# CORRECT?

CORRECT


In [8]:
%%bash
python3 tree-validity2.py << EOF
3
1 1 2
2 -1 -1
3 -1 -1
EOF
# INCORRECT?

INCORRECT


In [9]:
%%bash
python3 tree-validity2.py << EOF
3
2 1 2
1 -1 -1
2 -1 -1
EOF
# CORRECT?

CORRECT


In [10]:
%%bash
python3 tree-validity2.py << EOF
3
2 1 2
2 -1 -1
3 -1 -1
EOF
# INCORRECT?

INCORRECT


In [11]:
%%bash
python3 tree-validity2.py << EOF
1
2147483647 -1 -1
EOF
# CORRECT?

CORRECT


In [12]:
%%bash
python3 tree-validity2.py << EOF
5
30 1 2
20 -1 -1
50 3 4
30 -1 -1
60 -1 -1
EOF
# CORRECT?

CORRECT


In [13]:
%%bash
python3 tree-validity2.py << EOF
5
30 1 2
20 -1 -1
50 3 4
25 -1 -1
60 -1 -1
EOF
# INCORRECT?

INCORRECT
